In [1]:
!pip install transformers
!pip install sentencepiece
!pip install --upgrade accelerate
!pip install rouge.score nltk py7zr
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 58.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.4 MB/s eta 0:00:00
  Created wheel for rouge.score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=6db99add9105c29eaa2056be2db126677e15bd754

In [2]:
import transformers
from datasets import load_dataset, load_metric, load_from_disk ,DatasetDict
import numpy as np
import nltk
nltk.download('punkt')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data = load_dataset('cnn_dailymail', name='3.0.0',split=["train[:10000]","validation[:1000]","test[:1000]"])
data = DatasetDict({'train':data[0],'validation':data[1],'test':data[2]})


Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
metric = load_metric('rouge')

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ProphetNetForConditionalGeneration, ProphetNetTokenizer


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [6]:
# Load the tokenizer and model
tokenizer = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased',use_fast=False)
model = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased')


/opt/conda/lib/python3.10/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [7]:
max_input = 1024
max_target = 128
#tokenizer = transformers.PegasusTokenizer.from_pretrained(model_checkpoints)


def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [dialogue for dialogue in data_to_process['article']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['highlights'], max_length=max_target, padding='max_length', truncation=True)

  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs


tokenize_data = data.map(preprocess_data, batched = True, remove_columns=['id', 'article', 'highlights'])


  0%|          | 0/10 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
#import transformers
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)


#####################
# metrics
# compute rouge for evaluation
#####################

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}



In [14]:
args = transformers.Seq2SeqTrainingArguments(
    'cnndm-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size= 6,
    gradient_accumulation_steps=3,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=10,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )
#only CUDA available -> fp16=True
trainer = transformers.Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [15]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,2.082800,0.889108,34.668400,14.017100,24.470000,24.449000,56.131000
1,1.135100,0.877452,33.893400,14.200200,24.331800,24.323100,61.075000
3,0.996500,0.883048,34.902400,14.402700,24.867800,24.859700,61.545000
3,0.891100,0.900798,33.854000,13.787800,23.747900,23.735000,68.993000
4,0.799200,0.927996,34.281700,14.048300,23.944200,23.940400,69.251000
6,0.721300,0.947874,33.990600,13.591500,23.754700,23.730400,68.984000
6,0.666300,0.973562,34.087500,13.826300,23.887600,23.888900,69.849000
7,0.607800,0.989471,34.299400,13.772200,24.022500,24.012000,68.484000
9,0.539900,1.004761,34.521400,13.958900,24.069800,24.053400,69.273000
9,0.517400,1.013330,34.408700,13.945900,23.872000,23.834400,69.546000


TrainOutput(global_step=5550, training_loss=0.8627565352981155, metrics={'train_runtime': 29671.3564, 'train_samples_per_second': 1.685, 'train_steps_per_second': 0.187, 'total_flos': 5.50798553996329e+16, 'train_loss': 0.8627565352981155, 'epoch': 9.99})

In [18]:
predict_results = trainer.predict(
            tokenize_data['test'], metric_key_prefix="predict", max_length=128, num_beams=5
        )


In [19]:
predict_results.metrics

{'predict_loss': 1.0135927200317383,
 'predict_rouge1': 33.9214,
 'predict_rouge2': 13.4372,
 'predict_rougeL': 23.7567,
 'predict_rougeLsum': 23.7398,
 'predict_gen_len': 68.966,
 'predict_runtime': 1619.8468,
 'predict_samples_per_second': 0.617,
 'predict_steps_per_second': 0.206}

In [20]:
predictions = tokenizer.batch_decode(
                    predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )

In [21]:
import numpy as np
labels = np.where(predict_results.label_ids != -100, predict_results.label_ids, tokenizer.pad_token_id)
actual = tokenizer.batch_decode(
                    labels, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
from nltk.tokenize import sent_tokenize
preds = [pred.strip() for pred in predictions]
labels = [label.strip() for label in actual]

# rougeLSum expects newline after each sentence
preds = ["\n".join(sent_tokenize(pred)) for pred in preds] #predicted summary
labels = ["\n".join(sent_tokenize(label)) for label in labels] #actual summary

In [22]:
import pandas as pd
df = pd.DataFrame({"Article":data["test"]["article"],"Actual Summary":data["test"]["highlights"],'Model Summary':preds})
df.to_csv('outputProfetnetcnndm.csv',index=False)